# 1. Train Yolov11n using Ultralytics
First, use the pretrained model yolov11n to train the coco8 dataset.

In [ ]:
!pip install ultralytics
!pip install onnx
!pip install onnxruntime-gpu
!pip install onnxslim

In [ ]:
from ultralytics import YOLO
model = YOLO("yolo11n.pt")

In [ ]:
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

In [ ]:
results = model.predict("https://ultralytics.com/images/bus.jpg")

# 2. Export to ONNX format
After the training finish, convert the model to ONNX format. The yolov11n in the ONNX model will be use to convert to hailo format.

In [ ]:
model.export(
    format="onnx",
    imgsz=640,
    device=0,
    opset=11
)

In [ ]:
YOLO("/content/runs/detect/train/weights/best.onnx").predict("bus.jpg")

# 3. Setup the Hailo's Dataflow Compiler
This tool is to convert the ONNX model into the HEF format to be used on the Hailo8.

In [ ]:
!sudo apt-get update
!sudo apt install python3-pip
!sudo apt install python3.10-venv
!sudo apt-get install python3.10-dev python3.10-distutils python3-tk libfuse2 graphviz libgraphviz-dev
!sudo pip install pygraphviz
!sudo apt install wslu

In [ ]:
!python3.10 -m venv hdfc

## Important!

1. Download the .whl file from https://hailo.ai/developer-zone/software-downloads/
2. Choose Software Package: AI Software Suite -> Dataflow Compiler -> x86 -> Linux -> Python 3.10
3. Download the package the local and upload into this session runtime
4. Make sure the uploading is complete first before installing it



In [ ]:
!hdfc/bin/pip install /content/hailo_dataflow_compiler-3.31.0-py3-none-linux_x86_64.whl

In [ ]:
!hdfc/bin/hailo --version

# 4. Convert to Hailo format

## 4.1. Parse the ONNX to HAR

In [ ]:
with open("translate_model.py", "w") as f:
    f.write("""
from hailo_sdk_client import ClientRunner

# Define the ONNX model path and configuration
onnx_path = "/content/runs/detect/train/weights/best.onnx"
onnx_model_name = "yolov11n"
chosen_hw_arch = "hailo8"  # Specify the target hardware architecture

# Initialize the ClientRunner
runner = ClientRunner(hw_arch=chosen_hw_arch)

# Use the recommended end node names for translation for yolo
end_node_names = [
    "/model.23/cv2.0/cv2.0.2/Conv",
    "/model.23/cv3.0/cv3.0.2/Conv",
    "/model.23/cv2.1/cv2.1.2/Conv",
    "/model.23/cv3.1/cv3.1.2/Conv",
    "/model.23/cv2.2/cv2.2.2/Conv",
    "/model.23/cv3.2/cv3.2.2/Conv"
]
net_input_shapes={"input": [1, 3, 640, 640]},  # Adjust input shapes if needed

try:
    # Translate the ONNX model to Hailo's format
    hn, npz = runner.translate_onnx_model(
        onnx_path,
        onnx_model_name,
        end_node_names=end_node_names,
        # net_input_shapes=net_input_shapes,  # Adjust input shapes if needed
    )
    print("Model translation successful.")
except Exception as e:
    print(f"Error during model translation: {e}")
    raise

# Save the Hailo model HAR file
hailo_model_har_name = f"{onnx_model_name}.har"
try:
    runner.save_har(hailo_model_har_name)
    print(f"HAR file saved as: {hailo_model_har_name}")
except Exception as e:
    print(f"Error saving HAR file: {e}")
    """)

In [ ]:
!hdfc/bin/python translate_model.py

## 4.2.  Inspect the HAR

In [ ]:
with open("inspect_dict.py", "w") as f:
    f.write("""
from hailo_sdk_client import ClientRunner

# Load the HAR file
har_path = "yolov11n.har"

runner = ClientRunner(har=har_path)

from pprint import pprint

try:
    # Access the HailoNet as an OrderedDict
    hn_dict = runner.get_hn()  # Or use runner._hn if get_hn() is unavailable
    print("Inspecting layers from HailoNet (OrderedDict):")

    # Pretty-print each layer
    for key, value in hn_dict.items():
        print(f"Key: {key}")
        pprint(value)
        print("\\n" + "="*80 + "\\n")  # Add a separator between layers for clarity

except Exception as e:
    print(f"Error while inspecting hn_dict: {e}")
""")

In [ ]:
!hdfc/bin/python inspect_dict.py

## 4.3. Prepare the calibration data

In [ ]:
import numpy as np
from PIL import Image
import os
# from google.colab import drive

# Mounting Google Drive
# drive.mount('/content/drive/', force_remount=True)

# Paths to directories and files
image_dir = '/content/datasets/coco8/images/val'
output_dir = '/content/'
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# File paths for saving calibration data
calibration_data_path = os.path.join(output_dir, "calibration_data.npy")
processed_data_path = os.path.join(output_dir, "processed_calibration_data.npy")

# Initialize an empty list for calibration data
calib_data = []

# Process all image files in the directory
for img_name in os.listdir(image_dir):
    img_path = os.path.join(image_dir, img_name)
    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        img = Image.open(img_path).resize((640, 640))  # Resize to desired dimensions
        img_array = np.array(img) / 255.0  # Normalize to [0, 1]
        calib_data.append(img_array)

# Convert the calibration data to a NumPy array
calib_data = np.array(calib_data)

# Save the normalized calibration data
np.save(calibration_data_path, calib_data)
print(f"Normalized calibration dataset saved with shape: {calib_data.shape} to {calibration_data_path}")

# Scale the normalized data back to [0, 255]
processed_calibration_data = calib_data * 255.0

# Save the processed calibration data
np.save(processed_data_path, processed_calibration_data)
print(f"Processed calibration dataset saved with shape: {processed_calibration_data.shape} to {processed_data_path}")

## 4.4. Create the NMS config in json

In [ ]:
import json
import os

# Updated NMS layer configuration dictionary
nms_layer_config = {
    "nms_scores_th": 0.2,
    "nms_iou_th": 0.7,
    "image_dims": [
        640,
        640
    ],
    "max_proposals_per_class": 100,
    "classes": 80,
    "regression_length": 16,
    "background_removal": False,
    "background_removal_index": 0,
    "bbox_decoders": [
        {
            "name": "bbox_decoder51",
            "stride": 8,
            "reg_layer": "conv51",
            "cls_layer": "conv54"
        },
        {
            "name": "bbox_decoder62",
            "stride": 16,
            "reg_layer": "conv62",
            "cls_layer": "conv65"
        },
        {
            "name": "bbox_decoder77",
            "stride": 32,
            "reg_layer": "conv77",
            "cls_layer": "conv80"
        }
    ]
}

# Path to save the updated JSON configuration
output_dir = "/content/"
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
output_path = os.path.join(output_dir, "yolov11n_nms_config.json")

# Save the updated configuration as a JSON file
with open(output_path, "w") as json_file:
    json.dump(nms_layer_config, json_file, indent=4)

print(f"NMS layer configuration saved to {output_path}")



## 4.5. Optimize to HAR quantize

In [ ]:
with open("optimize_model.py", "w") as f:
    f.write("""
import os
from hailo_sdk_client import ClientRunner

# Define your model's HAR file name
model_name = "yolov11n"
hailo_model_har_name = f"{model_name}.har"

# Ensure the HAR file exists
assert os.path.isfile(hailo_model_har_name), "Please provide a valid path for the HAR file"

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=hailo_model_har_name)

# Define the model script to add a normalization layer
# Normalization for [0, 1] range
alls = \"\"\"
normalization1 = normalization([0.0, 0.0, 0.0], [255.0, 255.0, 255.0])
change_output_activation(conv54, sigmoid)
change_output_activation(conv65, sigmoid)
change_output_activation(conv80, sigmoid)
nms_postprocess("/content/yolov11n_nms_config.json", meta_arch=yolov8, engine=cpu)

allocator_param(width_splitter_defuse=disabled)
\"\"\"

# Load the model script into the ClientRunner
runner.load_model_script(alls)

# Define a calibration dataset
# Replace 'calib_dataset' with the actual dataset you're using for calibration
# For example, if it's a directory of images, prepare the dataset accordingly
calib_dataset = "/content/processed_calibration_data.npy"

# Perform optimization with the calibration dataset
runner.optimize(calib_dataset)

# Save the optimized model to a new Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

print(f"Quantized HAR file saved to: {quantized_model_har_path}")
    """)

In [ ]:
!hdfc/bin/python optimize_model.py

## 4.6. Compile the HAR quantize to HEF

In [ ]:
with open("compile_model.py", "w") as f:
    f.write("""
from hailo_sdk_client import ClientRunner

# Define the quantized model HAR file
model_name = "yolov11n"
quantized_model_har_path = f"{model_name}_quantized_model.har"

# Initialize the ClientRunner with the HAR file
runner = ClientRunner(har=quantized_model_har_path)
print("[info] ClientRunner initialized successfully.")

# Compile the model
try:
    hef = runner.compile()
    print("[info] Compilation completed successfully.")
except Exception as e:
    print(f"[error] Failed to compile the model: {e}")
    raise
file_name = f"{model_name}.hef"
with open(file_name, "wb") as f:
    f.write(hef)
    """)

In [ ]:
!hdfc/bin/python compile_model.py

In [ ]:
!hdfc/bin/hailo profiler yolov11n_quantized_model.har